In [ ]:
import chromadb
from chromadb.utils import embedding_functions
from utils import build_chroma_collection, prepare_car_reviews_data

In [ ]:
DATA_PATH = "data/archive/*"
CHROMA_PATH = "car_review_embeddings"
EMBEDDING_FUNC_NAME = "multi-qa-MiniLM-L6-cos-v1"
COLLECTION_NAME = "car_reviews"

In [ ]:
chroma_car_reviews_dict = prepare_car_reviews_data(DATA_PATH)

In [ ]:
build_chroma_collection(
    chroma_path=CHROMA_PATH,
    collection_name=COLLECTION_NAME,
    embedding_func_name=EMBEDDING_FUNC_NAME,
    ids=chroma_car_reviews_dict["ids"],
    documents=chroma_car_reviews_dict["documents"],
    metadatas=chroma_car_reviews_dict["metadatas"],
)

In [ ]:
client = chromadb.PersistentClient(CHROMA_PATH)

In [ ]:
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBEDDING_FUNC_NAME
)

In [ ]:
collection = client.get_collection(
    name=COLLECTION_NAME, embedding_function=embedding_func
)

In [ ]:
collection.count()

In [ ]:
great_reviews = collection.query(
    query_texts=["Find me some positive reviews that discuss the car's performance"],
    n_results=5,
    include=["documents", "distances", "metadatas"],
)

In [ ]:
print(great_reviews["documents"][0][0])

In [ ]:
from openai import OpenAI
import getpass

In [ ]:
KEY = getpass.getpass("give openai key:")

In [ ]:
context = "You are a customer success employee at a large car dealership."

question = "What is the key to great customer satisfaction?"

In [ ]:
openai_client = OpenAI(api_key=KEY)

In [ ]:
chat_response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": context},
        {"role": "user", "content": question},
    ],
    temperature=0,
    n=1,
)

In [ ]:
print(chat_response.choices[0].message.content)

In [ ]:
context = """
You are a customer success employee at a large
 car dealership. Use the following car reviews
 to answer questions: {}
"""

question = """
What's the key to great customer satisfaction
 based on detailed positive reviews?
"""

In [ ]:
good_reviews = collection.query(
    query_texts=[question],
    n_results=10,
    include=["documents"],
    where={"Rating": {"$gte": 3}},
)

reviews_str = ",".join(good_reviews["documents"][0])

In [ ]:
good_review_summaries = openai_client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": context.format(reviews_str)},
        {"role": "user", "content": question},
    ],
    temperature=0,
    n=1,
)

In [ ]:
print(good_review_summaries.choices[0].message.content)

In [ ]:
context = """
You are a customer success employee at a large car dealership.
Use the following car reviews to answer questions: {}
"""
question = """
Which of these poor reviews has the worst implications about our dealership?
 And explain why.
"""

In [ ]:
poor_reviews = collection.query(
    query_texts=[question],
    n_results=5,
    include=["documents"],
    where={"Rating": {"$lte": 3}},
)

reviews_str = ",".join(poor_reviews["documents"][0])

In [ ]:
poor_review_analysis = openai_client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": context.format(reviews_str)},
        {"role": "user", "content": question},
    ],
    temperature=0,
    n=1,
)

In [ ]:
print(poor_review_analysis.choices[0].message.content)